Link to Dataset: https://www.kaggle.com/datasets/aunanya875/suicidal-tweet-detection-dataset?resource=download&SSORegistrationToken=CfDJ8D29UYyoxK9CvcInYb4_m_sXuLPh9PlvatjblR1lO0nYvlqyf1EhRaTrF_e48shk4BZ8-bwXux7xNFFPIeploXGjO1MPBcHVV3pG9BScGKdzGDxRBEneKOLC26Ozi1YTJPsgq4N_5cfbH0ohFV0nj8_m8VtX1xFJrrgU8fh4rTGbsSu6jd8299SYkX7E08TAxp6ZokfQv3O7vbSAaxV-zoReAgcf5115V6uXreVKCxByQeHv6eKe2Y2RlA7DJVIg7BSMnzScb-cqOUOpjbLY8NGRB1UKzYOg-wNDqUJGd9vHSffHmTcRzlqHA4tWLTSpSEqCXvXOH4nJN5A24PZt56y3pyRG&DisplayName=Salome%20Chemiat

To do list:

*   Data Preparation / Preprocessing
*   Algorithm Selection


Bag Words
ANN
C-Bow
Transformers


*   Model Training
*   Model Evaluation


*   Analysis
*   Conclusion








Load Dummy File

Loading Dataset

In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Read the uploaded CSV file into a pandas DataFrame
file_name = list(uploaded.keys())[0]  # Extract the file name
data = pd.read_csv(file_name)

# Display the first few rows of the dataframe
data.head()



Saving suicide_tweets.csv to suicide_tweets.csv


,Tweet,Label
0,making some lunch,Not Suicide post
1,@Alexia You want his money.,Not Suicide post
2,@dizzyhrvy that crap took me forever to put to...,Potential Suicide post
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,Not Suicide post
4,Trying out &quot;Delicious Library 2&quot; wit...,Not Suicide post


In [ ]:
print("Class distribution in original dataset:")
print(data['Label'].value_counts())



Class distribution in original dataset:
Label
Not Suicide post           1127
Potential Suicide post      660
Name: count, dtype: int64


Using Undersampling we will eqaulize the two class labels so that Not Suicide will be fairly reduced to a similar value as Potential Suicide post

In [ ]:
# Check for any leading/trailing whitespaces in column labels
data.columns = data.columns.str.strip()

# Check for any missing values in the data
print("\nMissing values in the dataset:")
print(data.isnull().sum())

# Class distribution in the original dataset
print("\nClass distribution in the original dataset:")
print(data['Label'].value_counts())

# Clean the labels to ensure no hidden characters or spaces
data['Label'] = data['Label'].str.strip()

# Separate the data by class labels
class_1 = data[data['Label'] == 'Potential Suicide post']
class_0 = data[data['Label'] == 'Not Suicide post']

# Ensure that the minority class is used to determine the downsampling size
sample_size = len(class_1)  # Use the size of the minority class

# Perform undersampling to match the minority class size
class_0_undersampled = class_0.sample(sample_size, random_state=42)

# Concatenate the balanced classes
balanced_data = pd.concat([class_1, class_0_undersampled])

# Shuffle the dataset to avoid ordering bias
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the class distribution after balancing
print("\nClass distribution after balancing:")
print(balanced_data['Label'].value_counts())

# Check for any potential issues with the final dataset
print("\nFirst few rows of the balanced dataset:")
print(balanced_data.head())


Missing values in the dataset:
Tweet    2
Label    0
dtype: int64

Class distribution in the original dataset:
Label
Not Suicide post          1127
Potential Suicide post     660
Name: count, dtype: int64

Class distribution after balancing:
Label
Not Suicide post          660
Potential Suicide post    660
Name: count, dtype: int64

First few rows of the balanced dataset:
                                               Tweet                   Label
0                                      @gypsytrading        Not Suicide post
1  RT @Bob41Dylan: Happy birthday to the legendar...        Not Suicide post
2  becaus honestly, you did nothing wrong. i dont...  Potential Suicide post
3                  Scared of death, tired of living.  Potential Suicide post
4            Just woke up ...Good morning everyonee!        Not Suicide post


In [ ]:
print("Class distribution in original dataset:")
print(balanced_data['Label'].value_counts())
balanced_data.head()

Class distribution in original dataset:
Label
Not Suicide post          660
Potential Suicide post    660
Name: count, dtype: int64


,Tweet,Label
0,,Not Suicide post
1,rt happy birthday to the legendary neil young...,Not Suicide post
2,becaus honestly you did nothing wrong i dont k...,Potential Suicide post
3,scared of death tired of living,Potential Suicide post
4,just woke up good morning everyonee,Not Suicide post


Cleaning the dataset:
1. Data Cleaning

*   **Remove Special Characters:** Clean up HTML entities (e.g., &quot;) and character encoding artifacts (e.g., Ã¢Â€Â™).
*   **Normalize Text:** Convert text to lowercase and remove unnecessary punctuation.


*   **Remove URLs: **Strip links (e.g., https://t.co/...) as they don't contribute to meaning.
*   **Remove Mentions and Hashtags:** Replace @username and #hashtag with placeholders if necessary.


*   **Remove Stop Words (optional):** Eliminate common words like "and," "the," etc., depending on the chosen algorithm.
*    **Tokenization:** Split the text into meaningful units (tokens).








   

In [ ]:
import pandas as pd
import re

# Function to clean the tweet text
def clean_tweet(tweet):
    # Convert to string to handle non-string values
    tweet = str(tweet)
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)  # Remove URLs
    tweet = re.sub(r"@\w+|#\w+", '', tweet)  # Remove mentions and hashtags
    tweet = re.sub(r"[^A-Za-z0-9\s]", '', tweet)  # Remove special characters
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)  # Remove non-ASCII characters
    tweet = tweet.lower()  # Convert to lowercase
    return tweet

# Apply the cleaning function to the 'Tweet' column
balanced_data['Tweet'] = balanced_data['Tweet'].apply(clean_tweet)

# Check for missing values in the cleaned dataset
print("\nMissing values in the cleaned dataset:")
print(balanced_data.isnull().sum())

# Display the first few rows after cleaning
print("\nFirst few rows of the cleaned dataset:")
print(balanced_data.head())
balanced_data.to_excel('balanced_dataset.xlsx', index=False)

# Download the Excel file
files.download('balanced_dataset.xlsx')


Missing values in the cleaned dataset:
Tweet    0
Label    0
dtype: int64

First few rows of the cleaned dataset:
                                               Tweet                   Label
0                                                           Not Suicide post
1  rt  happy birthday to the legendary neil young...        Not Suicide post
2  becaus honestly you did nothing wrong i dont k...  Potential Suicide post
3                    scared of death tired of living  Potential Suicide post
4                just woke up good morning everyonee        Not Suicide post


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bag of Words (BoW) Technique with logistic Regression as a predicting model

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initializing the two columns of our dataset Text and Label
text_data = balanced_data['Tweet']
labels = balanced_data['Label']

# Split the data into training and testing sets, ensuring equal class distribution
X_train, X_test, y_train, y_test = train_test_split(
    text_data, labels, test_size=0.2, random_state=42, stratify=labels
)

# Apply Bag of Words (BoW) for vectorization - converting raw text data into a numerical format that machine learning algorithms can understand and process
vectorizer = CountVectorizer(stop_words='english')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_bow, y_train)

# Make predictions
y_pred = model.predict(X_test_bow)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Classification Report:
                        precision    recall  f1-score   support

      Not Suicide post       0.87      0.98      0.92       132
Potential Suicide post       0.97      0.85      0.91       132

              accuracy                           0.91       264
             macro avg       0.92      0.91      0.91       264
          weighted avg       0.92      0.91      0.91       264

Accuracy: 0.9129


Word2Vec Techniques with ANN

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from gensim.models import Word2Vec

# Ensuring that all necessary NLTK resources are downloaded
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Tokenize text data (this step can be skipped if data is already tokenized which was done in pre-processing)
balanced_data['Tokenized_Tweet'] = balanced_data['Tweet'].apply(word_tokenize)


In [ ]:
# Training Word2Vec model using the tokenized tweets
model_w2v = Word2Vec(sentences=balanced_data['Tokenized_Tweet'], vector_size=100, window=5, min_count=1, workers=4)

# Saving the model for future use
model_w2v.save("word2vec_model")
# Function to average word vectors for each tweet
def vectorize_tweet(tweet, model_w2v):
    vectors = [model_w2v.wv[word] for word in tweet if word in model_w2v.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model_w2v.vector_size)  # Return a zero vector if no words are in the model

# Applying the vectorization to the tokenized tweets
X_w2v = balanced_data['Tokenized_Tweet'].apply(lambda tweet: vectorize_tweet(tweet, model_w2v))

# Convert to an array format suitable for machine learning
X_w2v = np.array(X_w2v.tolist())
# Labels (0 = "Not Suicide post", 1 = "Potential Suicide post")
y = balanced_data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_w2v, y, test_size=0.2, random_state=42, stratify=y)
# Initialize and train the Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Logistic Regression Model Evaluation:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy:.4f}")



# Convert string labels to numerical labels
y_train = y_train.map({'Not Suicide post': 0, 'Potential Suicide post': 1}).astype(int)
y_test = y_test.map({'Not Suicide post': 0, 'Potential Suicide post': 1}).astype(int)


# Initialize the ANN model
ann_model = Sequential()

# Add input layer (the input shape corresponds to the size of Word2Vec vector)
ann_model.add(Dense(64, input_dim=X_w2v.shape[1], activation='relu'))

# Add hidden layer
ann_model.add(Dense(32, activation='relu'))

# Add output layer (binary classification)
ann_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
ann_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = ann_model.evaluate(X_test, y_test)
print(f'ANN Accuracy: {accuracy}')


Logistic Regression Model Evaluation:
                        precision    recall  f1-score   support

      Not Suicide post       0.69      0.72      0.71       132
Potential Suicide post       0.71      0.68      0.69       132

              accuracy                           0.70       264
             macro avg       0.70      0.70      0.70       264
          weighted avg       0.70      0.70      0.70       264

Accuracy: 0.7045
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5032 - loss: 0.6903 - val_accuracy: 0.5000 - val_loss: 0.6902
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5295 - loss: 0.6881 - val_accuracy: 0.5000 - val_loss: 0.6901
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5293 - loss: 0.6877 - val_accuracy: 0.6288 - val_loss: 0.6821
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6054 - loss: 0.6792 - val_accuracy: 0.6553 - val_loss: 0.6731
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6179 - loss: 0.6738 - val_accuracy: 0.5909 - val_loss: 0.6668
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5661 - loss: 0.6764 
ANN Accuracy: 0.5909090638160706


Transformer technique (particularly BERT (Bidirectional Encoder Representations from Transformers),

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm


# BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize and encode the text
def encode_data(texts, tokenizer, max_len=128):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')

# Encode the tweets
X = balanced_data['Tweet'].tolist()
y = balanced_data['Label'].tolist()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



# Encode the tweets
X = balanced_data['Tweet'].tolist()
# Convert labels to numerical format before creating the datasets
y = balanced_data['Label'].map({'Not Suicide post': 0, 'Potential Suicide post': 1}).tolist()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]) # Now labels should be numerical
        return item

    def __len__(self):
        return len(self.labels)



# Create datasets for training and testing
train_dataset = TweetDataset(train_encodings, y_train)
test_dataset = TweetDataset(test_encodings, y_test)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training Parameters
batch_size = 16
epochs = 3
learning_rate = 2e-5

# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())
# Evaluation Loop
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

# Print classification report
print("Transformer (BERT) Model Evaluation:")
print(classification_report(true_labels, predictions))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Transformer (BERT) Model Evaluation:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       132
           1       0.97      0.88      0.92       132

    accuracy                           0.92       264
   macro avg       0.93      0.92      0.92       264
weighted avg       0.93      0.92      0.92       264

